# 1. 라이브러리 불러오기 

In [2]:
# 데이터 처리 라이브러리
import os
import os.path as pth
from tqdm import tqdm
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

# Tensorflow 관련 라이브러리
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential,Model, load_model
from tensorflow.keras.layers import Conv2D, Dense, Dropout, BatchNormalization, Flatten, Add, Activation, Dense, Input
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[1],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3600)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

# 2. 데이터 불러오기 

## 2-1. Labeling 데이터프레임 불러오기 

In [6]:
# 학습 데이터 준비 
path = "./data/landmark"  
label_df = pd.read_csv(path + '/category.csv') # 각 랜드마크별 label
label_dict = dict(label_df[['landmark_name', 'landmark_id']].values)
label_dict_reverse = dict(label_df.values)

## 2-2. Train Data 경로 불러오기

In [7]:
## Train 파일(JPG)명과 label 정보를 담은 데이터 프레임 생성
train_dirs = path + '/train'
files = []
categories=[]
for img_dir in os.listdir(train_dirs):
    img_dir_list = train_dirs + '/' + img_dir
    
    for filename in os.listdir(img_dir_list):
        file_dir = img_dir + '/' + filename
        files.append(file_dir)
        categories.append(label_dict[img_dir])
            
train_data=pd.DataFrame(
                    {"file":files,
                    "label":categories}
                )    

label_dict[img_dir]

310

## 2-3. Test Data 경로 불러오기

In [8]:
# test 데이터 안 '6' 폴더에 있는 체크포인트 오류 데이터 제거 후 실행
test_dirs = path + '/test'
files = []
ids=[]
for img_cat in os.listdir(test_dirs):
    id_dir = test_dirs + '/' + img_cat
    for filename in os.listdir(id_dir):
        files.append(id_dir + '/' +filename)
        ids.append(filename.split('.JPG')[0])
                           
test_data = pd.DataFrame(
                    {"file":files,
                    "id":ids}
                )    

test_data

,file,id
0,./data/landmark/test/0/0hmnf5orki.JPG,0hmnf5orki
1,./data/landmark/test/0/0bgj9co0zl.JPG,0bgj9co0zl
2,./data/landmark/test/0/03123sl42g.JPG,03123sl42g
3,./data/landmark/test/0/0vwaki2su2.JPG,0vwaki2su2
4,./data/landmark/test/0/09jgq862fk.JPG,09jgq862fk
...,...,...
37959,./data/landmark/test/h/hf700dgjt3.JPG,hf700dgjt3
37960,./data/landmark/test/h/hf5vrn6vdx.JPG,hf5vrn6vdx
37961,./data/landmark/test/h/hr5xdtptl2.JPG,hr5xdtptl2
37962,./data/landmark/test/h/hj455fxmjr.JPG,hj455fxmjr


# 3. TFRecord 데이터 처리 

## 3-2. TFrecord 파일 TRAIN, VALIDATION, TEST(불러오기)

In [9]:
path = "./data/landmark" 
train_tfrecord_path = pth.join(path, 'tf_record_train.tfrecords')
valid_tfrecord_path = pth.join(path, 'tf_record_valid.tfrecords')

BUFFER_SIZE = 128
BATCH_SIZE = 16
NUM_CLASS = 1049

In [10]:
with tf.device('/device:GPU:1'):
    image_feature_description = {
        'image_raw': tf.io.FixedLenFeature([], tf.string),
        'randmark_id': tf.io.FixedLenFeature([], tf.int64),
        # 'id': tf.io.FixedLenFeature([], tf.string),
    }
    
def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)

def map_func(target_record):
    img = target_record['image_raw']
    label = target_record['randmark_id']
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label

def prep_func(image, label):
    result_image = image / 255
    result_image = tf.image.resize(result_image, (224,224))
    onehot_label = tf.one_hot(label, depth=NUM_CLASS)
    return result_image, onehot_label

with tf.device('/device:GPU:1'):
    dataset = tf.data.TFRecordDataset(train_tfrecord_path, compression_type='GZIP')
    dataset = dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    valid_dataset = tf.data.TFRecordDataset(valid_tfrecord_path, compression_type='GZIP')
    valid_dataset = valid_dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    valid_dataset = valid_dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    valid_dataset = valid_dataset.batch(BATCH_SIZE)
    valid_dataset = valid_dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [11]:
test_tfrecord_path = pth.join(path, 'tf_record_test.tfrecords')

with tf.device('/device:GPU:1'):
    test_image_feature_description = {
        'image_raw': tf.io.FixedLenFeature([], tf.string),
        'id': tf.io.FixedLenFeature([], tf.string),
    }
    
def test_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, test_image_feature_description)

def test_map_func(target_record):
    img = target_record['image_raw']
    label = target_record['id']
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label

def test_prep_func(image, label):
    result_image = image / 255.
   
    return result_image, label
    

with tf.device('/device:GPU:1'):    
    test_dataset = tf.data.TFRecordDataset(test_tfrecord_path, compression_type='GZIP')
    test_dataset = test_dataset.map(test_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.map(test_map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.shuffle(BUFFER_SIZE)
    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.map(test_prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    

    

# 4. 모델링 

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.applications import EfficientNetB5 

efficientnet = EfficientNetB5(weights ='imagenet', include_top = False, 
                              input_shape = (224, 224, 3), classifier_activation='softmax')

earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_loss", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=1e-7,
                        verbose=1)

model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장    
        filepath="./landmark_efficientnetb5.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)

print(efficientnet.summary())    
# print(#################################################################################)   

callbacks = [earlystop, learning_rate_reduction, model_check]

for layer in efficientnet.layers:
    layer.trainable = True
 
with tf.device('/device:GPU:1'):  
    model = models.Sequential()
    model.add(efficientnet)
    model.add(layers.Flatten())
    model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1049, activation='softmax')) 
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    
    history = model.fit(dataset,

                    epochs=100,
                    validation_data=valid_dataset,
                    callbacks = callbacks)

Model: "efficientnetb5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 224, 224, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 224, 224, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_1[0][0]            
_____________________________________________________________________________________

Total params: 28,513,527
Trainable params: 28,340,784
Non-trainable params: 172,743
__________________________________________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb5 (Functional)  (None, 7, 7, 2048)        28513527  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               51380736  
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
________________________________________________________________

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.applications import DenseNet121 

densenet = DenseNet121(weights ='imagenet', include_top = False, 
                              input_shape = (224, 224, 3))

earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_loss", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=1e-7,
                        verbose=1)

model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장    
        filepath="./landmark_densenet121.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)

# print(efficientnet.summary())    
# print(#################################################################################)   

callbacks = [earlystop, learning_rate_reduction, model_check]

for layer in densenet.layers:
    layer.trainable = True
 
with tf.device('/device:GPU:1'):  
    model = models.Sequential()
    model.add(densenet)
    model.add(layers.Flatten())
    model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1049, activation='softmax')) 
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    
    history = model.fit(dataset,
                    epochs=100,
                    validation_data=valid_dataset,
                    callbacks = callbacks)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 7, 7, 1024)        7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               25690624  
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1049)              538137    
Total params: 33,268,313
Trainable params: 33,183,641
Non-trainable params: 84,672
_____________________________________

In [12]:
from tensorflow.keras.models import load_model
densenet121_model = load_model("landmark_densenet121.h5")
densenet121_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 7, 7, 1024)        7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               25690624  
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1049)              538137    
Total params: 33,268,313
Trainable params: 33,183,641
Non-trainable params: 84,672
_____________________________________

In [ ]:
earlystop = EarlyStopping(patience=5)
learning_rate_reduction=ReduceLROnPlateau(
                        monitor= "val_loss", 
                        patience = 2, 
                        factor = 0.5, 
                        min_lr=1e-7,
                        verbose=1)

model_check = ModelCheckpoint( #에포크마다 현재 가중치를 저장    
        filepath="./landmark_densenet121.h5", #모델 파일 경로
        monitor='val_loss',  # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
        save_best_only=True)

# print(efficientnet.summary())    
# print(#################################################################################)   

callbacks = [earlystop, learning_rate_reduction, model_check]


with tf.device('/device:GPU:1'):  
    densenet121_model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(learning_rate=0.001),
                      metrics=['accuracy'])c


    history = densenet121_model.fit(dataset,
                        epochs=20,
                        validation_data=valid_dataset,
                        callbacks = callbacks)

Epoch 1/20
      2/Unknown - 1s 321ms/step - loss: 1.2285 - accuracy: 0.7812WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2505s vs `on_train_batch_end` time: 0.3897s). Check your callbacks.
4406/4406 [==============================] - 2311s 524ms/step - loss: 1.3006 - accuracy: 0.7167 - val_loss: 0.6044 - val_accuracy: 0.8606
Epoch 2/20
4406/4406 [==============================] - 2005s 455ms/step - loss: 0.7664 - accuracy: 0.8162 - val_loss: 0.4731 - val_accuracy: 0.8918
Epoch 3/20
4406/4406 [==============================] - 1538s 349ms/step - loss: 0.5763 - accuracy: 0.8561 - val_loss: 0.4172 - val_accuracy: 0.9036
Epoch 4/20
4406/4406 [==============================] - 1386s 315ms/step - loss: 0.4582 - accuracy: 0.8819 - val_loss: 0.4326 - val_accuracy: 0.8989
Epoch 5/20
1994/4406 [============>.................] - ETA: 11:51 - loss: 0.3937 - accuracy: 0.8966

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4406/4406 [==============================] - 3352s 761ms/step - loss: 0.1871 - accuracy: 0.9477 - val_loss: 0.2185 - val_accuracy: 0.9508
Epoch 7/20
4406/4406 [==============================] - 3359s 762ms/step - loss: 0.1361 - accuracy: 0.9612 - val_loss: 0.2077 - val_accuracy: 0.9535
Epoch 8/20
4406/4406 [==============================] - 2530s 574ms/step - loss: 0.1174 - accuracy: 0.9657 - val_loss: 0.1927 - val_accuracy: 0.9564
Epoch 9/20
4406/4406 [==============================] - 3270s 742ms/step - loss: 0.1032 - accuracy: 0.9696 - val_loss: 0.1995 - val_accuracy: 0.9568
Epoch 10/20
 309/4406 [=>............................] - ETA: 48:08 - loss: 0.0882 - accuracy: 0.9707